# Inline Scalar Functions

This is how most of us perceive a function. It is a user created piece of code to which we can pass parameters and obtain values consistently and with a known result.

For example, we could create a fullName function into which we pass a firstName and lastName and the function returns a fullName to us in a set format. Likewise we could even pass an employeeID to a function and obtain the employee's full name in a set and consistent format.

An Inline Scalar Function will only ever return a single value but can receive any number of defined parameters, including columns in order to produce the required output.

The format of the code is:

<code>CREATE FUNCTION <i>schema.functionName</i><br/>
(<br/>
    <i>@parameters parameterDataType</i><br/>
)<br/>
    RETURNS <i>dataTypeToBeReturned</i><br/>
AS<br/>
BEGIN<br/>
    RETURN<br/>
    (<br/>
        <i>functionCodeGoesHere</i><br/>
    )<br/>
END</code>

Sadly, as before, SQL Server does not allow us to create temporary functions therefore we will describe the syntax and the creation of functions, but will be able only to execute them rather than create our own from scratch in the demo database.

#### Simple Scalar Functions

Firstly, we will create our version of the T-SQL `GetDate()` function, called #fnGetDate in which we simply return the current date and time.

Using the syntax above we can create this as follows (noting that trying to execute the below will result in a permissions error):

In [ ]:
drop function if exists dbo.fnGetDate
go

create function dbo.fnGetDate()
returns datetime
as
begin
	return(select current_timestamp)
end
go

Note the use of <code>drop function if exists</code> in recognition that we are using a function.

We can also see from the above that we are creating a function that has no parameters passed to it.  However, syntax dictates that we must still call the function with empty brackets in order to avoid an error:

In [ ]:
select JupyterDatabase.dbo.fnGetDate();

As expected this returns the current date and time.

Using a Scalar Function in the SELECT line means the result of the function is applied to every record within a table.

Let's show this by adding the current date and time to each of the TOP 10 records from our hr.employee table:

In [ ]:
select top 10 employeeID, firstName, lastName, JupyterDatabase.dbo.fnGetDate() as currentTime
from JupyterDatabase.hr.employee
order by employeeID;

As mentioned in the definition above we can also pass parameters to functions.

We have therefore created a secondary function which allows us to pass in an INT value and the function will add that many days to the current date and time (provided for reference only):

In [ ]:
drop function if exists dbo.fnGetDateAdjust
go

create function dbo.fnGetDateAdjust
(
    @numDays int
)
returns datetime
as
begin
	return
    (
        select dateadd(dd, @numDays, current_timestamp)
    )
end
go

As you can see from the definition, we pass parameters in using the same syntax that we use for variables.  We place the @ symbol in front of the parameter name and then define its datatype.

We can then refer to this parameter within the main code of the function.  Hence we can utilise the parameter within <code>dateadd</code>.

Let's see this being used:

In [ ]:
select JupyterDatabase.dbo.fnGetDateAdjust(1) as nowPlus1Day,
        JupyterDatabase.dbo.fnGetDateAdjust(2) as nowPlus2Days;

In order to see scalar functions used in a more practical manner we will revisit our suggestion of a fullName function.

For this, we wish to pass in a first and last name and have the function return those two parameters combined into a single fullName result (provided for reference only):

In [ ]:
drop function if exists dbo.fnFullName
go

create function dbo.fnFullName
(
    @firstName varchar(25),
	@lastName varchar(35)
)
returns varchar(61)
as
begin
	return
    (
        select CONCAT_WS(' ', @firstName, @lastName)
    )
end
go

Note that we have altered the `return` value to be a varchar(61) in order to marry up with the value we expect to be returned (that being a string and with firstName at 25 characters, lastName at 35 characters, and the possibility of a space in betweeen).

Let's see this working with some manually passed in data:

In [ ]:
select JupyterDatabase.dbo.fnFullName('John', 'Smith');

Crucially, which we have not seen in the previous demos, we do not need to pass a hard coded value into a function, we can also pass in a column.

Considering this is a fullName function which expects first and last names it makes perfect sense that we should pass the firstName and lastName columns to our function.

We can do this as follows for the TOP 10 records from our hr.employee table:

In [ ]:
select top 10 employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullName(firstname, lastName) as fullName
from JupyterDatabase.hr.employee
order by employeeID;

This also satifies our requirement to have centralised and reusable code as we could now ensure that all developers use our `fnFullName` function in their reporting code and therefore if the need arose that we changed the definition, all code relying on the function would simply accommodate automatically.

However, do note that the same issue applies with performance as with Inline Functions in that we do not hide columns from SQL Server.

Therefore let's consider we are hoping to find someone with the fullName of 'Kevin Urquhart', we might expect to see someone code the following:

In [ ]:
select employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullName(firstname, lastName) as fullName
from JupyterDatabase.hr.employee
where JupyterDatabase.dbo.fnFullName(firstname, lastName) = 'Kevin Urquhart';

This, as before, causes SQL Server to evaluate every fullName in the table before returning any results.  Therefore consider refactoring to ensure performance is maintained:

In [ ]:
select employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullName(firstname, lastName) as fullName
from JupyterDatabase.hr.employee
where firstname = 'Kevin'
and lastName = 'Urquhart';

With this variant SQL Server can easily utilise any available indexes to search for either first or last names and satisfy the query with much less effort.

Even if indexes are not available, it is always best to code in this manner and with this in mind as your DBA may well choose to add indexes in the future based on your query patterns.

#### Table Based Scalar Functions

Finally in this section we have Table Based Scalar Functions.

Previously we have passed in values that can be used directly by the function to return a value (for example, first and last names can simply be combined to return a full name), but what if we wish to pass in employeeID instead?

In this scenario we would need the function to use the employeeID to access the hr.employee table and then return the relevant full name.  Hence these are Table Based as they rely on table access to fulfil their purpose.

Let's have a look at a fullName function using this premise (again, for reference only as this code will not execute on the demo database):

In [ ]:
drop function if exists dbo.fnFullNameTable
go

create function dbo.fnFullNameTable
(
    @employeeID int
)
returns varchar(61)
as
begin
	return
    (
        select CONCAT_WS(' ', firstName, lastName)
		from JupyterDatabase.hr.employee
		where employeeID = @employeeID
    )
end
go

Based on the functions we have seen before it's easy to see that this works in the same manner.

We pass in an employeeID and the function has code which utilises that parameter in order to obtain a single return value.

Hence we can use the same query as above except that, instead of first and last names being passed in, we change these to employeeID:

In [ ]:
select top 10 employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullNameTable(employeeID) as fullName
from JupyterDatabase.hr.employee
order by employeeID;

#### Performance Notes

We have already discussed how functions can impact the use of Indexes within SQL Server and this is equally valid in regard to Inline Scalar Functions.

However, there is another consideration we need to know when attempting to use Table Based Scalar Functions which is that SQL Server will evaluate the function individually for each record of the table.

Consider the following scenario:

-   We have an employee table of 1 million records
-   We have no indexes on our employee table
-   We wish to return every record in the table alongside its fullName

This gives us a couple of options for our queries:

In [ ]:
-- Option 1

select employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullName(firstname, lastName) as fullName
from JupyterDatabase.hr.employee;

-- Option 2

select employeeID, firstName, lastName,
        JupyterDatabase.dbo.fnFullNameTable(employeeID) as fullName
from JupyterDatabase.hr.employee;

These will, as we know, both yield the same and correct results.

The next consideration should be performance:

<u>Query One</u>

This will read the hr.employee table once in entirety.

For each record it will read the firstName and lastName, the function will concatenate them, and the record will be returned.

Therefore, for a 1 million record table, SQL Server will read 1 million records in order to obtain our answer.

<u>Query Two</u>

As we have no indexes on our employee table, for SQL Server to satisfy <code>WHERE employeeID = @employeeID</code> it will need to read every record in our table.

This means that for every row SQL Server needs to read the row itself and then 1 million rows to look for the correct employeeID in order to return a full name.

The results of this maths are astronomical (1 million lots of 1 million and 1 records) and therefore, even without listing a very large number here, it's immediately apparent that Table Based Scalar Functions can have a very negative impact on performance.  These, therefore, need to be used wisely.